In [1]:
import pandas as pd

In [2]:
df_nn = pd.read_csv("cmoa_no_null.csv")
df_nn.head()

,title,creation_date,creation_date_earliest,creation_date_latest,medium,accession_number,id,credit_line,date_acquired,department,...,party_type,full_name,cited_name,role,nationality,birth_date,death_date,birth_place,death_place,small_img_url
0,Keith Haring,1984,1984-01-01,1984-01-01,gelatin silver print,2002.17,cmoa:things/692a68c5-af1e-4124-80f1-cbf38be51abe,Milton Fine Fund,2002-06-06,Contemporary Art,...,Person,Robert Mapplethorpe,"Mapplethorpe, Robert",NaN,American,1947-01-01,1989-01-01,"New York City (New York state, United States)","New York City (New York state, United States)",https://cmoa-collection-images.s3.amazonaws.co...
1,Untitled,1964-1965,1964-01-01,1965-01-01,oil on canvas,2002.19,cmoa:things/871505af-a072-4a84-ab09-3abf932951b8,Joseph Soffer Family Trust Fund,2002-06-06,Contemporary Art,...,Person,Jo Baer,"Baer, Jo",NaN,American,1929-01-01,NaN,"Seattle (King County, Washington, United States)",NaN,https://cmoa-collection-images.s3.amazonaws.co...
2,Snuff bottle: Carved Glass,late 18th century,1770-01-01,1799-01-01,glass,79.H,cmoa:things/fd7a0764-d061-4de5-9820-7076510037aa,Gift of Howard Heinz,1938-12-28,Decorative Arts and Design,...,Person,unknown Chinese,unknown Chinese,NaN,Chinese,NaN,NaN,NaN,NaN,https://cmoa-collection-images.s3.amazonaws.co...
3,Snuff bottle: Carved Glass,late 19th century,1875-01-01,1899-01-01,glass,77.H,cmoa:things/25d8d23b-03a7-480f-ab1d-d36b5d01f543,Gift of Howard Heinz,1938-12-28,Decorative Arts and Design,...,Person,unknown Chinese,unknown Chinese,NaN,Chinese,NaN,NaN,NaN,NaN,https://cmoa-collection-images.s3.amazonaws.co...
4,Snuff bottle: Carved Glass,18th century,1700-01-01,1799-01-01,glass,80.H,cmoa:things/4d8941d0-b89f-4750-a538-71e0037417d8,Gift of Howard Heinz,1938-12-28,Decorative Arts and Design,...,Person,unknown Chinese,unknown Chinese,NaN,Chinese,NaN,NaN,NaN,NaN,https://cmoa-collection-images.s3.amazonaws.co...


In [3]:
df_nn.columns

Index(['title', 'creation_date', 'creation_date_earliest',
       'creation_date_latest', 'medium', 'accession_number', 'id',
       'credit_line', 'date_acquired', 'department', 'physical_location',
       'item_width', 'item_height', 'item_depth', 'item_diameter', 'web_url',
       'provenance_text', 'classification', 'image_url', 'artist_id',
       'party_type', 'full_name', 'cited_name', 'role', 'nationality',
       'birth_date', 'death_date', 'birth_place', 'death_place',
       'small_img_url'],
      dtype='object')

In [18]:
df_names = df_nn["full_name"].value_counts()
df_names = df_names[ ~df_names.index.str.contains("unknown")]
df_names

full_name
Charles "Teenie" Harris    466
Samuel Rosenberg           225
Hiroshige Andō             212
W. Eugene Smith            160
Clyde Hare                 135
                          ... 
Thomas J. Doyle              1
Jean René Bazaine            1
Hans Albert Falk             1
David Levine                 1
George Inness                1
Name: count, Length: 3528, dtype: int64

In [20]:
df_filtered = df_nn[ df_nn["full_name"].isin(df_names[:50].index) ]
df_filtered.head()

,title,creation_date,creation_date_earliest,creation_date_latest,medium,accession_number,id,credit_line,date_acquired,department,...,party_type,full_name,cited_name,role,nationality,birth_date,death_date,birth_place,death_place,small_img_url
14,Hôtel de la Duchesse de Villars--rue des Saint...,1901-1902,1901-01-01,1902-01-01,albumen print,2002.24.2.3,cmoa:things/bff5fb1b-1e33-479c-a9e5-acd61d07c526,Second Century Acquisition Fund,2002-06-06,Photography,...,Person,Eugène Atget,"Atget, Eugène",NaN,French,1857-01-01,1927-01-01,"Libourne (Gironde, Aquitaine, France)",Paris,https://cmoa-collection-images.s3.amazonaws.co...
15,Ancien Monastère des Bénédictins anglais--269 ...,1905,1905-01-01,1905-01-01,albumen print,2002.24.2.5,cmoa:things/95c2efac-cbeb-4886-8fe0-dd97ad675a2b,Second Century Acquisition Fund,2002-06-06,Photography,...,Person,Eugène Atget,"Atget, Eugène",NaN,French,1857-01-01,1927-01-01,"Libourne (Gironde, Aquitaine, France)",Paris,https://cmoa-collection-images.s3.amazonaws.co...
20,"Fontaine du Montreuil, faubourg Saint-Antoine ...",1904-1905,1904-01-01,1905-01-01,albumen print,2002.24.2.4,cmoa:things/f46e92ad-427a-4262-9bfa-ad06af9da8ef,Second Century Acquisition Fund,2002-06-06,Photography,...,Person,Eugène Atget,"Atget, Eugène",NaN,French,1857-01-01,1927-01-01,"Libourne (Gironde, Aquitaine, France)",Paris,https://cmoa-collection-images.s3.amazonaws.co...
25,"Rue (du Chevalier) de la Barre-Montmarte, 18e ...",1899-1900,1899-01-01,1900-01-01,albumen print,2002.24.2.1,cmoa:things/5575373f-33e5-43ee-8214-a77383762d93,Second Century Acquisition Fund,2002-06-06,Photography,...,Person,Eugène Atget,"Atget, Eugène",NaN,French,1857-01-01,1927-01-01,"Libourne (Gironde, Aquitaine, France)",Paris,https://cmoa-collection-images.s3.amazonaws.co...
26,"Cabaret de l'Épée de Bois - ou fut assassiné, ...",1901,1901-01-01,1901-01-01,albumen print,2002.24.2.2,cmoa:things/50a2526c-08a2-4b35-97f6-3b0f4aa9e43d,Second Century Acquisition Fund,2002-06-06,Photography,...,Person,Eugène Atget,"Atget, Eugène",NaN,French,1857-01-01,1927-01-01,"Libourne (Gironde, Aquitaine, France)",Paris,https://cmoa-collection-images.s3.amazonaws.co...


In [ ]:
df_filtered.to_csv("cmoa_vis2.csv")